In [ ]:
# !pip install carball

In [1]:
import carball
import pandas as pd
import json

In [2]:
# loading in data
analysis_manager = carball.analyze_replay_file('B51EAD91453AC7D38125D19096C534ED.replay')

Replay name not found
Found bot not in bot list
Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 7781 by flour
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


the source of the above replay file can be found here: \
https://ballchasing.com/replay/50e81ea9-caaf-44bb-a4c5-12829a6ed123#watch?t=261.12s \
notice the accidental own goal at frame 7781, at 4:26 \
note that own-goals will be coutned as a non-shot goal 

In [3]:
# loading data into json
json_object = analysis_manager.get_json_data()

In [4]:
# getting a list of all the frames with shots 
shots = []
for hit in json_object['gameStats']['hits']:
    if 'shot' in hit.keys():
        shots.append(hit['frameNumber'])
shots

[1139, 1320, 1766, 1776, 3030, 3116, 5665, 6050, 6689, 7140, 7362, 8771, 9454]

In [5]:
# getting a list of all the frames with goals
goals = []
for hit in json_object['gameStats']['hits']:
    if 'goal' in hit.keys():
        goals.append(hit['frameNumber'])
goals

[3116, 6050, 6689, 7781, 8771, 9454]

In [6]:
# parsing the json file and extracting data about the players and to which team they belong

teamNumber = [0,0,1,1]
teamIdentifier = [0,1,0,1]
player_id = []
for team in json_object['teams']:
    player_id.append(team['playerIds'][0]['id'])
    player_id.append(team['playerIds'][1]['id'])
    
team_id = pd.DataFrame({'team': teamNumber, 
                        'teammate_identifier': teamIdentifier,
                        'player_id': player_id})

team_id

,team,teammate_identifier,player_id
0,0,0,76561199019631043
1,0,1,b13f4235beb
2,1,0,b8cec16e75b
3,1,1,76561198255906637


In [7]:
# parsing the json file and extracting data about the players and their names

player_id = []
player_name = []
for player in json_object['players']:
    player_id.append(player['id']['id'])
    player_name.append(player['name'])
id_name = pd.DataFrame({'player_id': player_id, 
                        'name': player_name})
id_name

,player_id,name
0,76561199019631043,flour
1,b8cec16e75b,Weebrahim001
2,b13f4235beb,thewrongending
3,76561198255906637,MEQONATOR


In [8]:
# merging the two dataframes we created above
id_name_team = id_name.merge(team_id, how = 'inner', on = 'player_id')
id_name_team

,player_id,name,team,teammate_identifier
0,76561199019631043,flour,0,0
1,b8cec16e75b,Weebrahim001,1,0
2,b13f4235beb,thewrongending,0,1
3,76561198255906637,MEQONATOR,1,1


In [9]:
# creating columns to denote who is whose opponent
# not the most elegant solution
id_name_team['opp_1'] = list(id_name_team.loc[[1,2,3,0],'name'])
id_name_team['opp_2'] = list(id_name_team.loc[[3,0,1,2],'name'])
id_name_team['team_mate'] = list(id_name_team.loc[[2,3,0,1],'name'])
id_name_team

,player_id,name,team,teammate_identifier,opp_1,opp_2,team_mate
0,76561199019631043,flour,0,0,Weebrahim001,MEQONATOR,thewrongending
1,b8cec16e75b,Weebrahim001,1,0,thewrongending,flour,MEQONATOR
2,b13f4235beb,thewrongending,0,1,MEQONATOR,Weebrahim001,flour
3,76561198255906637,MEQONATOR,1,1,flour,thewrongending,Weebrahim001


In [23]:
# create a dataframe where each row represents a frame where a shot took place
# the 'player' column denotes the shottaker
frames = [hit['frameNumber'] for hit in json_object['gameStats']['hits']]
shottaker = [hit['playerId']['id'] for hit in json_object['gameStats']['hits']]
frames_shottaker = pd.DataFrame({'index': frames,
                                 'frame': frames,
                                 'shot_taker_id': shottaker})
frames_shottaker.set_index('index', inplace = True)
frames_shottakers = frames_shottaker.loc[shots]
frames_shottakers

,frame,shot_taker_id
index,,
1139,1139,76561198255906637
1320,1320,b13f4235beb
1766,1766,b8cec16e75b
1776,1776,b8cec16e75b
3030,3030,76561198255906637
3116,3116,b8cec16e75b
5665,5665,76561199019631043
6050,6050,b13f4235beb
6689,6689,b8cec16e75b


In [24]:
# merging this dataframe with the id_name_team dataframe we created above
frames_shottakers = frames_shottakers.merge(id_name_team, how = 'outer', left_on = 'shot_taker_id', right_on = 'player_id').sort_values(by = 'frame').reset_index()
frames_shottakers.rename(columns={'name':'shot_taker'},inplace=True)

# cleaning up columns we no longer need
frames_shottakers.drop(columns = ['index','player_id','team','teammate_identifier'],inplace = True)
frames_shottakers

,frame,shot_taker_id,shot_taker,opp_1,opp_2,team_mate
0,1139,76561198255906637,MEQONATOR,flour,thewrongending,Weebrahim001
1,1320,b13f4235beb,thewrongending,MEQONATOR,Weebrahim001,flour
2,1766,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR
3,1776,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR
4,3030,76561198255906637,MEQONATOR,flour,thewrongending,Weebrahim001
5,3116,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR
6,5665,76561199019631043,flour,Weebrahim001,MEQONATOR,thewrongending
7,6050,b13f4235beb,thewrongending,MEQONATOR,Weebrahim001,flour
8,6689,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR
9,7140,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR


In [25]:
# from the replay file, gather a dataframe
# each row represents a frame of the game 
# the dataframe is unfortunately multi-indexed, making it hard to work with
gamedata = analysis_manager.get_data_frame()

In [26]:
# limit the dataframe to only frames where shots were made
gamedata = gamedata.loc[shots]

In [27]:
# create a column that is True if the shot resulted in a goal 
gamedata['goals'] = [True if r in goals else False for r in gamedata.index]

### Translating data
The xG calculations need the data to be oriented from the "point of view" of the shot-taker, so we need to clean the information so that it is translated to this format

In [39]:
# create a deep copy of the above dataframe
# so that it's easier to re-run when messed up
result = frames_shottakers.copy('deep')

In [28]:
# helper function to get information from the multi-index dataframe
def getter(dataframe, col_name, shot_taker_name):
    return dataframe[shot_taker_name][col_name]

In [51]:
# formula that helps us re-orient the data
# example input is "shot_taker" and "pos_x"
# the function will use previously created dataframes to find who is the shot_taker in every frame
# and grab the correct person's pos_x 

def reorient(person_title, column_name):
    output = []
    
    for frame in frames_shottakers.frame:
        index = frames_shottakers[frames_shottakers.frame == frame].index[0]
        name = frames_shottakers[person_title][index]
        output.append(getter(gamedata.loc[frame],column_name, name))
    
    return output

In [52]:
# creates a new column in the given dataframe, using the reorienting function above
def create_columns(df, title, column_name):
    new_col_name = title + "_" + column_name
    df[new_col_name] = reorient(title, column_name)

In [53]:
# creates a new column in the given dataframe, using the reorienting function above
# specific to the columns applicable to the ball's physics
def create_ball_columns(df, column_name):
    new_col_name = 'ball' + "_" + column_name
    df[new_col_name] = list(gamedata['ball'][column_name])

In [70]:
# reorient columns applicable to players
column_names = gamedata[gamedata.columns[0][0]].columns
titles = ['shot_taker', 'opp_1', 'opp_2'] #excluding teammate for now

for col in column_names:
    for title in titles:
        create_columns(result, title, col)

In [71]:
# reorient columns applicable to balls
ball_col_names = gamedata['ball'].columns
for col in ball_col_names:
    create_ball_columns(result, col)

In [80]:
# add a column to represent whether shot resulted in a goal
result['goal'] = list(gamedata['goals'])
result

,frame,shot_taker_id,shot_taker,opp_1,opp_2,team_mate,shot_taker_ping,opp_1_ping,opp_2_ping,shot_taker_pos_x,...,ball_vel_y,ball_vel_z,ball_ang_vel_x,ball_ang_vel_y,ball_ang_vel_z,ball_hit_team_no,ball_rot_x,ball_rot_y,ball_rot_z,goal
0,1139,76561198255906637,MEQONATOR,flour,thewrongending,Weebrahim001,18.0,9.0,6.0,-3177.110107,...,24271.000977,-2614.100037,-2333.200073,-2489.600067,732.799988,0.0,0.515783,-0.204240,1.530124,False
1,1320,b13f4235beb,thewrongending,MEQONATOR,Weebrahim001,flour,6.0,18.0,6.0,-218.350006,...,-26069.699707,1550.899963,1593.699951,-4404.599915,1899.799957,1.0,-0.607505,0.963578,1.221102,False
2,1766,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR,6.0,6.0,9.0,2262.290039,...,15404.399414,4032.799988,-550.900002,962.900009,-966.800003,0.0,-0.140437,-0.317772,1.549792,False
3,1776,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR,6.0,6.0,9.0,1941.319946,...,17157.600098,7096.799927,-943.600006,192.800007,-1000.899963,0.0,-0.033897,-0.637878,1.241377,False
4,3030,76561198255906637,MEQONATOR,flour,thewrongending,Weebrahim001,18.0,9.0,6.0,-2358.600098,...,15143.000488,5180.100098,2993.900146,3895.199890,-3444.299927,0.0,0.010523,-2.604407,-1.288792,False
5,3116,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR,6.0,6.0,9.0,-355.549988,...,13634.499512,-2637.099915,726.699982,3705.799866,-4662.399902,0.0,1.006334,-0.097924,1.500355,True
6,5665,76561199019631043,flour,Weebrahim001,MEQONATOR,thewrongending,9.0,6.0,18.0,723.789978,...,-27862.700195,9337.600098,-3775.700073,-2688.699951,-3809.700012,1.0,0.671171,-2.675260,-1.918426,False
7,6050,b13f4235beb,thewrongending,MEQONATOR,Weebrahim001,flour,6.0,19.0,6.0,744.010010,...,-18248.699951,-212.299995,-871.500015,-799.000015,-5882.299805,1.0,-1.164805,1.071936,1.955957,True
8,6689,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR,6.0,6.0,9.0,704.039978,...,16946.300049,7119.600220,2716.000061,-1618.999939,-2968.500061,0.0,0.513060,1.347475,-0.621864,True
9,7140,b8cec16e75b,Weebrahim001,thewrongending,flour,MEQONATOR,6.0,6.0,9.0,-1907.800049,...,14775.000000,6965.900269,1549.700012,-862.500000,-4029.299927,0.0,0.134227,0.176815,1.279224,False
